In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv('movie_reviews.csv.bz2', compression='bz2')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [3]:
# take a peek at the data
print(dataset.head())
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [4]:
# build train and test datasets
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

In [5]:
#pre Processing
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(train_reviews)

In [6]:
train_sequences = t.texts_to_sequences(train_reviews)

In [7]:
test_sequences = t.texts_to_sequences(test_reviews)

In [8]:
print("Vocabulary size={}".format(len(t.word_counts)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=105664
Number of Documents=35000


In [10]:
MAX_SEQUENCE_LENGTH = 500
# pad dataset to a maximum review length in words
X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [11]:
le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0

In [12]:
y_train = le.fit_transform(train_sentiments)
y_test = le.transform(test_sentiments)

In [13]:
VOCAB_SIZE = len(t.word_counts)+1

In [14]:
#Prepare the Mode
EMBED_SIZE = 32
EPOCHS=5
BATCH_SIZE=128

In [15]:
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           3381280   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               2000250   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 5,384,885
T

In [16]:
# Fit the model
model.fit(X_train, y_train, 
          validation_split=0.2,
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          verbose=1)


Train on 28000 samples, validate on 7000 samples
Epoch 1/5
28000/28000 [==============================] - 53s 2ms/step - loss: 0.4178 - accuracy: 0.7816 - val_loss: 0.2558 - val_accuracy: 0.8974
Epoch 2/5
28000/28000 [==============================] - 49s 2ms/step - loss: 0.1373 - accuracy: 0.9501 - val_loss: 0.2720 - val_accuracy: 0.8969
Epoch 3/5
28000/28000 [==============================] - 48s 2ms/step - loss: 0.0403 - accuracy: 0.9883 - val_loss: 0.3719 - val_accuracy: 0.8883
Epoch 4/5
28000/28000 [==============================] - 47s 2ms/step - loss: 0.0098 - accuracy: 0.9981 - val_loss: 0.4421 - val_accuracy: 0.8873
Epoch 5/5
28000/28000 [==============================] - 48s 2ms/step - loss: 0.0022 - accuracy: 0.9998 - val_loss: 0.5254 - val_accuracy: 0.8883


In [17]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 89.32%


In [19]:
predictions = model.predict_classes(X_test)
predictions[:10]

array([[0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1]])

In [20]:
predictions = ['positive' if item == 1 else 'negative' for item in predictions]
predictions[:10]

['negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive']

In [21]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(predictions ,test_sentiments))
print(classification_report(predictions ,test_sentiments))

[[6679  791]
 [ 811 6719]]
             precision    recall  f1-score   support

   negative       0.89      0.89      0.89      7470
   positive       0.89      0.89      0.89      7530

avg / total       0.89      0.89      0.89     15000

